In [1]:
#Para crear y análizar redes
import networkx as nx
#Para leer desde un archivo
import pandas as pd 
#Para generar errores aleatorios
import random
# using datetime module
import datetime
import os

In [2]:
def imprimir_hora():
    # ct stores current time
    ct = datetime.datetime.now()
    print("current time:-", ct)

In [3]:
class Red:
    #inicializar los atributos 
    def __init__(self, n, t, na):
        self.nombre = n
        self.topologia = t
        self.num_ataques = na

        # NÚMERO DE COMPONENTES CONECTADOS
        self.ncc_aleatorio = list()
        self.ncc_degree = list()
        self.ncc_closeness = list()
        self.ncc_harmonic = list()
        self.ncc_pagerank = list()
        self.ncc_betweenness = list()

        # PROPORCIÓN DEL COMPONENTE GIGANTE
        self.pcg_aleatorio = list()
        self.pcg_degree = list()
        self.pcg_closeness = list()
        self.pcg_harmonic = list()
        self.pcg_pagerank = list()
        self.pcg_betweenness = list()

        # DIÁMETRO
        self.diametro_aletorio = list()
        self.diametro_degree = list()
        self.diametro_closeness = list()
        self.diametro_harmonic = list()
        self.diametro_pagerank = list()
        self.diametro_betweenness = list()

        # LONGITUD PROMEDIO DE LA TRAYECTORIA
        self.aspl_aleatorio = list()
        self.aspl_degree = list()
        self.aspl_closeness = list()
        self.aspl_harmonic = list()
        self.aspl_pagerank = list()
        self.aspl_betweenness = list()

        # PROBABILIDAD DE CONEXIÓN
        self.pc_aleatorio = list()
        self.pc_degree = list()
        self.pc_closeness = list()
        self.pc_harmonic = list()
        self.pc_pagerank = list()
        self.pc_betweenness = list()

        # NODO ELIMINADO
        self.nodo_aleatorio = list()
        self.nodo_degree = list()
        self.nodo_closeness = list()
        self.nodo_harmonic = list()
        self.nodo_pagerank = list()
        self.nodo_betweenness = list()

    
    ### M E T R I C A S ###

    def num_componentes_conectados(self, G):
        #A la variable num_cc se le asigna el valor de componentes conectados.
        print("Componentes conectados")
        imprimir_hora()
        num_cc = nx.number_connected_components(G)
        return num_cc
    
    def prop_componente_gigante(self, G):
        print("Proporcion del componente gigagnte")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True) # Duda, por qué los ordena?
        G0 = G.subgraph(componentes_conectados[0]) #Componente Gigant    
        tcg = len(G0) #Tamaño del componente Gigante, cuenta los elementos de la lista G0
        n = len(nx.nodes(G)) # Cuenta el numero de nodos que tiene la red G
        pcg = tcg / n #Proporción del componente gigante
        return pcg

    def diametro(self, G):
        print("Diametro")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(componentes_conectados[0])
        diameter = nx.diameter(G0)
        return diameter

    def trayectoria_promedio(self, G):
        print("Trayectoria promedio")
        imprimir_hora()
        componentes_conectados = sorted(nx.connected_components(G), key=len, reverse=True)
        G0 = G.subgraph(componentes_conectados[0])
        aspl = nx.average_shortest_path_length(G0) #longitud de ruta más corta promedio
        return aspl

    def probabilidad_de_conexion(self, G):
        print("Probabilidad de conexión")
        imprimir_hora()
        componentes = sorted(nx.connected_components(G))
        sumar_componentes = []
        sumar_probabilidad = []
        for x in componentes:
            y = (len(x))
            sumar_componentes.append(y)
            sumar_probabilidad.append((y * (y - 1))/2)
        resultado = sum(sumar_componentes)
        if resultado > 1:
            probabilidad = sum(sumar_probabilidad) / ((resultado * (resultado - 1))/ 2)
        else:
            probabilidad = 0
        return probabilidad

    
    ### A T A Q U E S ###    
    def centrality(self, G, tipo):
        H = G.copy()
        if tipo == "aleatorio":
            nodo_remover = random.sample(H.nodes, 1) #Elegir un nodo aleatorio
            #print(nodo_remover)
        elif tipo == "grado":
            max_centrality = max(H.degree, key=lambda x: x[1])[0]
            nodo_remover = [max_centrality]
        elif tipo == "cercania":
            centralidad = nx.closeness_centrality(H) #Hacer la centralidad de cada nodo (diccionario)
        elif tipo == "armonico":
            centralidad = nx.harmonic_centrality(H) #Hacer la centralidad de cada nodo (diccionario) 
        elif tipo == "pagerank":
            centralidad = nx.pagerank(H) #Por defecto el maximo de iteraciones es 100
        elif tipo == "intermediacion":
            centralidad = nx.betweenness_centrality(H) #Hacer la centralidad de cada nodo (diccionario)
        if tipo == "cercania" or tipo =="armonico" or tipo == "pagerank" or tipo == "intermediacion":
            max_centrality = (max(centralidad, key=centralidad.get)) #Encontrar la llave con el valor más alto
            nodo_remover = [max_centrality]
        
        H.remove_nodes_from(nodo_remover) #Remover el nodo
        
        return (H, nodo_remover)


    ### FUNCIÓN TIPO DE ATAQUE ###
    def ataque(self, tipo):
        G = self.topologia.copy()
        num_cc_lista = [1] # Número de componentes conectados
        prop_cg_lista = [1] # Proporción del componente gigante
        ataques_lista = [0]
        ataque = 0
        diametro_lista = [self.diametro(G)] # Diámetro        
        longitud_promedio = [self.trayectoria_promedio(G)] # Longitud de la trayectoria Promedio
        probabilidad_conexion = [self.probabilidad_de_conexion(G)] # Probabilidad de conexión de nodos
        nodo_eliminado = [0] #Inicia con 0 nodos

        for x in range(self.num_ataques):
            print("*** Número de ataque: ", x, " ***")
            G, n_eliminado = self.centrality(G, tipo) #########################################3
            num_cc_lista.append(self.num_componentes_conectados(G))
            prop_cg_lista.append(self.prop_componente_gigante(G))
            ataque = ataque + 1 # ataque += 1
            ataques_lista.append(ataque/len(nx.nodes(self.topologia))) # Proporción de nodos eliminados   
            diametro_lista.append(self.diametro(G)) # Lista Diámetro    
            probabilidad_conexion.append(self.probabilidad_de_conexion(G)) #Lista probabilidad de conexión
            longitud_promedio.append(self.trayectoria_promedio(G)) # Lista Trayectoria Promedio
            nodo_eliminado.append(n_eliminado)

        if tipo == "aleatorio": 
            self.ncc_aleatorio = num_cc_lista
            self.pcg_aleatorio = prop_cg_lista
            self.diametro_aletorio = diametro_lista
            self.aspl_aleatorio = longitud_promedio
            self.pc_aleatorio = probabilidad_conexion
            self.nodo_aleatorio = nodo_eliminado
        
        elif tipo == "grado":
            self.ncc_degree = num_cc_lista
            self.pcg_degree = prop_cg_lista
            self.diametro_degree = diametro_lista
            self.aspl_degree = longitud_promedio
            self.pcg_degree = probabilidad_conexion
            self.nodo_degree = nodo_eliminado
        
        elif tipo == "cercania":
            self.ncc_closeness = num_cc_lista
            self.pcg_closeness = prop_cg_lista
            self.diametro_closeness = diametro_lista
            self.aspl_closeness = longitud_promedio
            self.pc_closeness = probabilidad_conexion
            self.nodo_closeness = nodo_eliminado
        
        elif tipo == "armonico":
            self.ncc_harmonic = num_cc_lista
            self.pcg_harmonic = prop_cg_lista
            self.diametro_harmonic = diametro_lista
            self.aspl_harmonic = longitud_promedio
            self.pc_harmonic = probabilidad_conexion
            self.nodo_harmonic = nodo_eliminado
        
        elif tipo == "pagerank":
            self.ncc_pagerank = num_cc_lista
            self.pcg_pagerank = prop_cg_lista
            self.diametro_pagerank = diametro_lista
            self.aspl_pagerank = longitud_promedio
            self.pc_pagerank = probabilidad_conexion
            self.nodo_pagerank = nodo_eliminado
        
        elif tipo == "intermediacion":
            self.ncc_betweenness = num_cc_lista
            self.pcg_betweenness = prop_cg_lista
            self.diametro_betweenness = diametro_lista
            self.aspl_betweenness = longitud_promedio
            self.pc_betweenness = probabilidad_conexion
            self.nodo_betweenness = nodo_eliminado
        
        return ataques_lista, num_cc_lista, prop_cg_lista, diametro_lista, longitud_promedio, probabilidad_conexion, nodo_eliminado

In [4]:
def atacar_red(grafo, nombre_grafo, num_ataques, ataque):
    r1 = Red(nombre_grafo, grafo, num_ataques)
    # declarar un diccionario por cada metrica
    diametro = dict()
    num_cc = dict()
    proporcion_cg = dict()
    aspl = dict()
    prob_conexion = dict()
    nodo_eliminar = dict()
    

    #tipos_ataques = ["aleatorio", "grado", "cercania", "harmonica", "pagerank", "intermediacion"]

  # Cambio
    #for ataque in tipos_ataques:
        #print("*************************")
    print("Tipo de ataque: " + ataque)
    #imprimir_hora()
    ataques_lista, num_cc_lista, prop_cg_lista, diametro_lista, longitud_promedio, probabilidad_conexion, nodo_eliminado = r1.ataque(ataque)
    #Almacenar las metricas en sus diccionarios
    diametro["diametro"] = diametro_lista
    num_cc["numero_cc"] = num_cc_lista
    proporcion_cg["proporcion_cg"] = prop_cg_lista
    aspl["aspl"] = longitud_promedio
    prob_conexion["probabilidad_conexion"] = probabilidad_conexion
    nodo_eliminar["nodo_eliminado"] = nodo_eliminado

    #Agregar la lista de ataques a cada diccionario
    diametro["num_ataques"] = ataques_lista
    num_cc["num_ataques"] = ataques_lista
    proporcion_cg["num_ataques"] = ataques_lista
    aspl["num_ataques"] = ataques_lista
    prob_conexion["num_ataques"] = ataques_lista
    nodo_eliminar["num_ataques"] = ataques_lista

    #Almacenar cada diccionario (son 6) en un csv
    pd.DataFrame(diametro).to_csv('resultados\\diametro\\' + nombre_grafo + '_' + str(ataque) + '_diametro.csv', index=False)
    pd.DataFrame(num_cc).to_csv('resultados\\numero_cc\\' + nombre_grafo + '_' + str(ataque) + '_num_cc.csv', index=False)
    pd.DataFrame(proporcion_cg).to_csv('resultados\\proporcion_cg\\' + nombre_grafo + '_' + str(ataque) + '_proporcion_cg.csv', index=False)
    pd.DataFrame(aspl).to_csv('resultados\\aspl\\' + nombre_grafo + '_' + str(ataque) + '_aspl.csv', index=False)
    pd.DataFrame(prob_conexion).to_csv('resultados\\prob_conexion\\' + nombre_grafo + '_' + str(ataque) + '_prob_conexion.csv', index=False)
    pd.DataFrame(nodo_eliminar).to_csv('resultados\\nodos_eliminados\\' + nombre_grafo + '_' + str(ataque) + '_nodo_eliminar.csv', index=False)

In [5]:
folder_resultados = "resultados"
folders = ['diametro', 'numero_cc', 'proporcion_cg', 'aspl', 'prob_conexion', 'nodos_eliminados']

if not os.path.exists(folder_resultados + "\\"):
    os.makedirs(folder_resultados)

for folder in folders:
    if not os.path.exists(folder_resultados + "\\" + folder + "\\"):
        os.makedirs(folder_resultados + "\\" + folder)

In [6]:
# (Regla, Enlace, Experimento, Tipo de ataque)
sredes= [(1,16,7,"grado"),(1,32,9,"grado"),
        (2,1,3,"grado"),(2,2,7,"grado"),(2,4,10,"grado"),(2,8,9,"grado"),(2,16,2,"grado"),(2,32,8,"grado"),
        (1,1,3,"cercania"),(1,2,4,"cercania"),(1,4,2,"cercania"),(1,8,1,"cercania"),(1,16,5,"cercania"),(1,32,3,"cercania"),
        (2,1,3,"cercania"),(2,2,7,"cercania"),(2,4,6,"cercania"),(2,8,9,"cercania"),(2,16,9,"cercania"),(2,32,10,"cercania"),
        (1,1,3,"intermediacion"),(1,2,3,"intermediacion"),(1,4,7,"intermediacion"),(1,8,6,"intermediacion"),(1,16,7,"intermediacion"),(1,32,6,"intermediacion"),
        (2,1,3,"intermediacion"),(2,2,7,"intermediacion"),(2,4,6,"intermediacion"),(2,8,10,"intermediacion"),(2,16,1,"intermediacion"),(2,32,8,"intermediacion"),
        (1,1,3,"armonico"),(1,2,4,"armonico"),(1,4,2,"armonico"),(1,8,1,"armonico"),(1,16,8,"armonico"),(1,32,1,"armonico"),
        (2,1,3,"armonico"),(2,2,7,"armonico"),(2,4,6,"armonico"),(2,8,9,"armonico"),(2,16,1,"armonico"),(2,32,4,"armonico"),
        (1,1,3,"pagerank"),(1,2,3,"pagerank"),(1,4,10,"pagerank"),(1,8,1,"pagerank"),(1,16,1,"pagerank"),(1,32,3,"pagerank"),
        (2,1,3,"pagerank"),(2,2,2,"pagerank"),(2,4,6,"pagerank"),(2,8,9,"pagerank"),(2,16,7,"pagerank"),(2,32,9,"pagerank")]

In [7]:
redes = [(2,1,3,"grado"),(2,2,7,"grado"),(2,4,10,"grado"),(2,8,9,"grado"),(2,16,2,"grado"),(2,32,8,"grado"),
        (1,1,3,"cercania"),(1,2,4,"cercania"),(1,4,2,"cercania"),(1,8,1,"cercania"),(1,16,5,"cercania"),(1,32,3,"cercania"),
        (2,1,3,"cercania"),(2,2,7,"cercania"),(2,4,6,"cercania"),(2,8,9,"cercania"),(2,16,9,"cercania"),(2,32,10,"cercania"),
        (1,1,3,"intermediacion"),(1,2,3,"intermediacion"),(1,4,7,"intermediacion"),(1,8,6,"intermediacion"),(1,16,7,"intermediacion"),(1,32,6,"intermediacion"),
        (2,1,3,"intermediacion"),(2,2,7,"intermediacion"),(2,4,6,"intermediacion")]
for red in redes:
    nombre_grafo = "r" + str(red[0]) + "_enlaceD" + str(red[1]) + "_experimento" + str(red[2]) + "_ciclo30.graphml"
    ruta_grafo = "graphs\\r" + str(red[0]) + "\\enlaceD" + str(red[1]) + "\\experimento" + str(red[2]) + "\\" + nombre_grafo

    grafo = nx.read_graphml(ruta_grafo)
    print("Nombre grafo: " + nombre_grafo)
    print("Ruta grafo: " + ruta_grafo)

    atacar_red(grafo, "r" + str(red[0]) + "_enlaceD" + str(red[1]) + "_experimento" + str(red[2]) + "_ciclo30", 2499, str(red[3]))


Nombre grafo: r1_enlaceD8_experimento6_ciclo30.graphml
Ruta grafo: graphs\r1\enlaceD8\experimento6\r1_enlaceD8_experimento6_ciclo30.graphml
Tipo de ataque: grado
Diametro
current time:- 2023-05-11 23:12:37.559577
Trayectoria promedio
current time:- 2023-05-11 23:14:22.336315
Probabilidad de conexión
current time:- 2023-05-11 23:16:00.202178
*** Número de ataque:  0  ***
Componentes conectados
current time:- 2023-05-11 23:16:00.263015
Proporcion del componente gigagnte
current time:- 2023-05-11 23:16:00.272988
Diametro
current time:- 2023-05-11 23:16:00.281962
Probabilidad de conexión
current time:- 2023-05-11 23:18:05.069270
Trayectoria promedio
current time:- 2023-05-11 23:18:05.084890
*** Número de ataque:  1  ***
Componentes conectados
current time:- 2023-05-11 23:20:10.436598
Proporcion del componente gigagnte
current time:- 2023-05-11 23:20:10.442582
Diametro
current time:- 2023-05-11 23:20:10.451559
Probabilidad de conexión
current time:- 2023-05-11 23:21:44.147027
Trayectoria pr